In [ ]:
# Imports and init remote
import os
import subprocess
from union import workflow, FlyteFile, UnionRemote, ImageSpec, Resources, FlyteDirectory, ActorEnvironment, Artifact
from flytekit.configuration import Config

os.environ["UNION_CONFIG"] = "/Users/pryceturner/.union/config_serving.yaml"

remote = UnionRemote(
    config=Config.auto(config_file=os.environ["UNION_CONFIG"]), 
    default_domain="development", 
    default_project="flytesnacks"
    )

In [ ]:
# Define Image
image = ImageSpec(
    name="boltz",
    packages=[
        "union",
        "flytekit==1.15",
        "union-runtime==0.1.11",
        "fastapi==0.115.11",
        "pydantic==2.10.6",
        "uvicorn==0.34.0",
        "python-multipart==0.0.20",
    ],
    apt_packages=["build-essential"],
    builder="union",
    commands=["pip install boltz==0.4.1"]
)

In [ ]:
actor = ActorEnvironment(
    name="boltz-actor",
    replica_count=1,
    ttl_seconds=600,
    requests=Resources(
        cpu="2",
        mem="10Gi",
        gpu="1",
    ),
    container_image=image,
)

In [ ]:

@actor.task
def simple_predict(input: FlyteFile) -> FlyteDirectory:
    input.download()
    out = "/tmp/boltz_out"
    os.makedirs(out, exist_ok=True)
    subprocess.run(["boltz", "predict", input.path, "--out_dir", out, "--use_msa_server"])
    return FlyteDirectory(path=out)

@workflow
def act_wf(input: FlyteFile) -> FlyteDirectory:
    return simple_predict(input=input)

execution = remote.execute(
    entity=act_wf, 
    inputs={"input": "inputs/boltz/prot_no_msa.yaml"}, 
    wait=True
)
output = execution.outputs
print(output)

In [ ]:
%%writefile boltz_fastapi.py
import os
import io
import shutil
import asyncio
import tempfile
import traceback
import subprocess
from pathlib import Path
from typing import Optional, Dict, Any
from click.testing import CliRunner
from boltz.main import predict
from fastapi import FastAPI, File, UploadFile, Form, BackgroundTasks
from fastapi.responses import JSONResponse, StreamingResponse

app = FastAPI()

def package_outputs(output_dir: str) -> bytes:
    import io
    import tarfile

    tar_buffer = io.BytesIO()
    parent_dir = Path(output_dir).parent

    cur_dir = os.getcwd()
    with tarfile.open(fileobj=tar_buffer, mode="w:gz") as tar:
        os.chdir(parent_dir)
        try: 
            tar.add(Path(output_dir).name, arcname=Path(output_dir).name)
        finally: 
            os.chdir(cur_dir)

    return tar_buffer.getvalue()

async def generate_response(process, out_dir, yaml_path):
    try:
        while True:
            try:
                stdout, stderr = await asyncio.wait_for(process.communicate(), timeout=10.0)
                break
            except TimeoutError:
                yield b""  # Yield null character instead of spaces

        if process.returncode != 0:
            raise Exception(stderr.decode())

        print(stdout.decode())

        # Package the output directory
        tar_data = package_outputs(f"{out_dir}/boltz_results_{Path(yaml_path).with_suffix('').name}")
        yield tar_data

    except Exception as e:
        traceback.print_exc()
        yield JSONResponse(status_code=500, content={"error": str(e)}).body

@app.post("/predict/")
async def predict_endpoint(
    yaml_file: UploadFile = File(...),
    msa_file: Optional[UploadFile] = File(None),
    options: Optional[Dict[str, str]] = Form(None)
):
    yaml_path = f"/tmp/{yaml_file.filename}"
    with open(yaml_path, "wb") as buffer:
        shutil.copyfileobj(yaml_file.file, buffer)

    msa_path = f"/tmp/{msa_file.filename}"
    with open(msa_path, "wb") as buffer:
        shutil.copyfileobj(msa_file.file, buffer)

    # Create a temporary directory for the output
    with tempfile.TemporaryDirectory() as out_dir:
        # Call boltz.predict as a CLI tool
        try:
            print(f"Running predictions with options: {options} into directory: {out_dir}")
            # Convert options dictionary to key-value pairs
            options_list = [f"--{key}={value}" for key, value in (options or {}).items()]
            if msa_file and os.path.exists(msa_path):
                print(f"MSA file included at {msa_path}")
            else:
                options_list.append("--use_msa_server")
            command = ["boltz", "predict", yaml_path, "--out_dir", out_dir, "--cache", "/tmp/.boltz_cache"] + options_list
            print(f"Running command: {' '.join(command)}")
            process = await asyncio.create_subprocess_exec(
                *command,
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE
            )
            return StreamingResponse(generate_response(process, out_dir, yaml_path), media_type="application/gzip", headers={"Content-Disposition": f"attachment; filename=boltz_results.tar.gz"})

        except Exception as e:
            traceback.print_exc()
            return JSONResponse(status_code=500, content={"error": str(e)})


In [ ]:
from datetime import timedelta

from union import Resources, ImageSpec
from union.app import App, ScalingMetric, Input
from flytekit.extras.accelerators import GPUAccelerator

boltz_model = Artifact(
    project="flytesnacks",
    domain="development",
    name="boltz-1",
    version="7c1d83b779e4c65ecc37dfdf0c6b2788076f31e1",
    partitions={
        "task": "auto",
        "model_type": "custom",
        "huggingface-source": "boltz-community/boltz-1",
        "format": "None",
        "architecture": "custom",
        "_u_type": "model",
    },
)

boltz_fastapi = App(
    name="boltz-fastapi-notebook",
    container_image=image,
    inputs=[
        Input(
            name="boltz_model", value=boltz_model.query(), download=True, mount="/tmp/.boltz_cache"
        ),
    ],
    limits=Resources(cpu="2", mem="10Gi", gpu="1", ephemeral_storage="50Gi"),
    port=8080,
    include=["./boltz_fastapi.py"],
    args=["uvicorn", "boltz_fastapi:app", "--host", "0.0.0.0", "--port", "8080"],
    env={
        "PYTORCH_ENABLE_MPS_FALLBACK": "1",
    },
    min_replicas=1,
    max_replicas=3,
    scaledown_after=timedelta(minutes=10),
    scaling_metric=ScalingMetric.RequestRate(1),
    accelerator=GPUAccelerator("nvidia-l40s"),
)

from union.remote._app_remote import AppRemote

app_remote = AppRemote(default_project="default", default_domain="development", union_remote=remote)

app_remote.deploy(boltz_fastapi)

In [ ]:
%%writefile prot.yaml
version: 1
sequences:
  - protein:
      id: A
      sequence: QLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCG
      msa: /tmp/seq.a3m


In [ ]:
%%writefile seq.a3m
>101
SDQLEDSEVEAVAKGLEEXYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAXISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>UniRef100_A0A8S0JH07	200	0.549	9.920E-56	4	134	136	45	171	173
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIAFAENELETNINNPY----DRVPWDEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGM-
>UniRef100_UPI00110EAFB6	199	0.977	2.558E-55	0	135	136	34	169	170
SDQLEDSEVEAVAKGLEEMYTNGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>UniRef100_UPI001CF7A1DD	185	0.539	1.616E-50	9	134	136	1	122	124
---------DDVANVLDQMFRNGVNEKNFTEYVYKNFSQKDIALAENKLETNINN----LYDRVPWDEMGGCIARKIREEFFAMTNVSLIVKYAQKKAWLELAKVVLRFVKANGLKTNTYIIAGQLAIWAVQCGL-
>UniRef100_A0A455TWS1	177	0.686	8.943E-48	1	134	136	35	168	170
-EQIKNSEVDTVAQGLEQMFSNGVSEENFKNYVNANFSSEEITKSEKELDVNLSNTSSPIQARVNWNGLGQCMANKIKDEFFAMINVGAIVAAAQKKAWKELAMTVLIFAKANGLKTNALIVAGQLAVWAVQCGL-
>UniRef100_UPI0012B3E02F	177	0.466	8.943E-48	4	134	136	44	176	179
----EQRQIDEVAAVLEKMFADGVTEENLKQYAQANYSEEELIIADNELNTNLSQIQDEnaIMYKVDWGALGNCMANKIKDELLAMISVGTIIKYAQKKAWKELAKIVIKYVAKAGVKTNAALIAGQLAIWGLQCGI-
>UniRef100_A0A5Q2SBQ8	172	0.476	5.426E-46	9	134	136	1	128	131
---------DEVAAVLEKMFADGVTEENLKQYAQANYSEEELIIADNELNTNLSQIQDEnaIMYKVDWGALGNCMANKIKDELLAMISVGTIIKYAQKKAWKELAKIVIKYVAKAGVKTNAALIAGQLAIWGLQCGI-
>UniRef100_A0A7U9RNI7	166	0.503	6.190E-44	5	134	136	43	181	185
-----DPEVAVVAQELEKIFANGVSQENLNKYVLKNFSNKELTVAEKELDVNYNpfslqskndNNSLHSVSVYGWNNLGQCMYNKIKDEFFEMVNIGVIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVAAQLAVWAVQCGM-
>UniRef100_UPI00062A1226	162	0.286	1.456E-42	10	134	136	55	183	186
----------AVAKELEKMFVDGDIENLNIDYLIAKYGKDEIQATERFIGISENEsrifPTEHRIVQRDLADIGNCMLGKLGEEIRSMVNVNTIVAYIDKKLWLEAAKAIVAKVAAQGIKRNAAVMATVLAWYAVQCGL-
>UniRef100_UPI0013568C51	162	0.445	2.738E-42	9	134	136	1	128	131
---------DKFAAVLEKMFAKGVTEENFKQYAQANYSEEELMIADREMNTNLSqiQDDDVIMYKMDWNALGSYMANKIKDELLAMISIGTIITYAKRKAWKELATIVIKYVAKAWVRTNVAFIAGQLAIWGLQCGI-
>UniRef100_UPI001FD87A89	159	0.533	1.821E-41	4	134	136	39	171	173
----EDKEIELVANELEAMFSNGVTENNLKNYVNENYDSRDISTAEKELNTSLkkTNYNYLLRSKFSWNKFGNCMVNEIKDEFFAMINVATIVKYAKKKSWKKLAGVVLKFAKANGLKTNVAIIAGQLAVWAIKCGI-
>UniRef100_UPI001A0EA69D	158	0.598	4.696E-41	3	134	136	44	173	175
---ISSQEVDQVAQALELMFDNNVSTSNFKKYVNNNFSDSEIAIAELELESRISN--SRSEFRVAWNEMGGCIAGKIRDEFFAMISVGTIVKYAQKKAWKELALVVLKFVKANGLKTNAIIVAGQLALWAVQCGL-
>UniRef100_UPI000DA36189	158	0.617	6.440E-41	7	134	136	48	173	175
-------EVDQVAQALELMFDNNVSTSNFKKYVNNNFSDSEIAIAELELESRISN--SRSEFRVAWNEMGGCIAGKIRDEFFAMISVGTIVKYAQKKAWKELALVVLKFVKANGLKTNAIIVAGQLALWAVQCGL-
>UniRef100_E1LSW0	157	0.518	1.211E-40	19	126	136	1	104	106
-------------------FRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPY----DRVPWDEMGGCIAGKIREDFFAMTNVSLIVKYAQKRAWLELAKVVLRFVKANRLKTNIYIIAGQLA---------
>UniRef100_A0A556UDB4	155	0.471	5.872E-40	0	134	136	34	173	177
SDSVQSQDVDTVATELQKMFAHGVSQENLNKYAKENFSKQELQAASRELDVNYLANtssttYSPFISMFSWNSMGKCMYNKIKDELFAMVNVGVIVKYAKKKAWKELAKVVIRVAKGAGVKTNAILVAGQLAVWAVACGM-
>UniRef100_A0A081PZB4	133	0.444	2.802E-32	4	93	136	75	160	161
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALSENELETNINNPY----DRVPWDEMGGCIAGKIRDEFFAMINVSLIVKYA------------------------------------------
>UniRef100_K0Z545	115	0.231	4.125E-26	37	132	136	144	251	263
-------------------------------------SSEQLALIVNSLNGAADNPYVDAQSDRSWGSFGSCVVTGVLGFSPFQIDYNLLGKYIYEKSWKKVADLLKRYAKKEIAKKSGNIVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI000D25C0E4	108	0.213	1.654E-23	23	132	136	90	210	222
-----------------------VNEKLAAEYGFGR-SSEQLALIVNSLNGTSENPYVDAQSVRSWGSFGSCVVTGVLGFSPFQIDYNLLGKYIYEKSWNKVAGLLKRYAKKEIANKGGNIVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI001C8F0653	107	0.493	3.107E-23	57	129	136	23	95	100
---------------------------------------------------------DVIMYKMDWNALGSYMANKIKDELLAMISIGTIITYAKRKAWKELATIVIKYVAKAWVRTNVAFIAGQLAIWG------
>UniRef100_UPI000D52D7A3	103	0.210	5.309E-22	17	132	136	87	210	222
-----------------KFFVD---EKLAAEYGFGR-SSEQLALIVNSLNGTSEHPHIDAQSVRSWGSFGSCVVTGVLGFSPFQIDYNLLGKYIYEKAWNKVAGLLKRYAKKEIAKKGGNIVLKQiikstpvgfaawLGVYAVGC---
>UniRef100_A0A133S201	103	0.386	7.277E-22	19	93	136	1	71	72
-------------------FRNGVNEKNFTECVYKNFSQKDIALAENKLETNINN----LYDRVPWDEMGGCIARKIREEFFAMTNVSLTVRYA------------------------------------------
>UniRef100_UPI000B18BA61	97	0.666	8.234E-20	72	134	136	0	62	66
------------------------------------------------------------------------MYNKIKDEFFAMVNIEVIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVTAQLAVWAVQCGM-
>UniRef100_UPI00207479F9	93	0.244	1.924E-18	1	132	136	16	154	158
-EQIQDSqnQVDKVAKEFEDLFTNGiqINENSYainSDYLVQNYSSEEISGIVSLIEESSLINNTSTRSKRDIGSFLVCMKDKAVGDLKDMFNVGKFLVFIKTKAWKQAAEFAVKWLAKNGVKRNAVATAALLGWYGVQC---
>UniRef100_A0A2N8LAA9	93	0.259	1.924E-18	3	132	136	36	170	174
---ISQSEVDAVAIEFEKLFSNGIIisGNNYSinyDYLNNNYTSEEIQAFINLMASSELSPISSGRRKRSISSFVVCMKDKAVSDIADMFKVSAFVSFVQRKAWKEAAKFAVSWLAKNGIKRNVAATAALLSWYGIQC---
>UniRef100_UPI001FF59259	88	0.233	8.429E-17	5	132	136	30	162	166
-----QTEIESVASEFEQLFTKGIIisGNNYTfnhDYLTNNYTSDEIQAFIHLMDSTDLSPTFSKRRKRSIGSFAVCMKDKAVSDIADMFKVGAFVSFIQRKAWKEAAKFAVSWLAKNGIKRNVAATAALLSWYGIQC---
>UniRef100_UPI000407185C	87	0.745	2.168E-16	84	134	136	1	51	53
------------------------------------------------------------------------------------TNVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>UniRef100_UPI00135BC820	83	0.255	3.685E-15	5	132	136	38	170	174
-----QAEVDAVAVELENLFTngiliNGDSYTLNIDYLYKNYTPEEVDSFIALMKSSNLSSNTINRKKRSVNSFLVCMKDKAVADIADMFKVGAFVSFIQRKAWREAAQFAVSWLARNGIKRNVAATAALLSWYGVQC---
>UniRef100_A0A2N6UQ44	79	0.181	1.173E-13	3	132	136	47	200	206
---ITEAEIDAFSQDVADKFVilmrdgiiyDGSSFRSNDDVLSAAGFANEADLIEElvdSLNKGANNRYTSVYTDRSLSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEKAWNKVADLLQKYAKKEIEKKAGSVVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI001C0EBA32	79	0.168	1.607E-13	3	132	136	48	201	213
---ITEAEIDAFSQEVADKFVilmrdgiiyDGSSFRSNDDVLSAmGFSNEAdlIKELVDSLNNGANNRYANVYTERSLSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEKAWNKVADLLQKYAKKEIKKKAGSVVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI001C0F1009	78	0.162	4.127E-13	3	132	136	47	200	212
---ITEAEIDAFSQEVADKFVilmrdgiiyDGSSFRSNDDVLSAmGFSNEAdlIKELVDSLNNGANNRYANVYTERSLSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEKAWNKVADLLQKYAKKEIKKKAGSVLLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI000214F415	66	0.346	2.716E-09	5	53	136	24	72	93
-----DPEVAIAAQELEKIFVDVVSQKNLNRYALKNFSNKELTVAEKELDVNYN----------------------------------------------------------------------------------
>UniRef100_UPI0012EE93E6	58	0.238	2.655E-06	62	124	136	108	170	202
--------------------------------------------------------------DRSWASFGECVVTGILGFSIFGIDYNLLGQYIKQKAWGAAAALLKKEAEKELKKQGAKIALKQ-----------
>UniRef100_UPI0011BE0601	57	0.141	6.774E-06	3	96	136	48	153	158
---ITEAEIDAFSQEVADKFVilmrdgiiyDGSSFRSNDDVLSAmGFSNgaDLIKELVDSLNNGANNRYANVYTERSWSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEK---------------------------------------
>UniRef100_UPI000370F184	55	0.201	2.359E-05	29	132	136	98	206	211
-----------------------------KEVLISNLGYDEASATAESLGLTIDELVSDRTVpntngDRGVGEFLSCMKGHATDDLKSIFNVNAVSYYIGKEKYFEAALAAGKHLAKQGIRRNAYGLIGILAWYGVRC---
>UniRef100_A0A0G3H5S3	55	0.206	3.222E-05	4	132	136	51	180	185
----QEEQVEALA-ELLKEMDEAGSEAEVQQLFVERFGQDNLEDAAAQLGTNTSEVFreSDEVVEEGFGEFLQCIKGKATNDIKKALDVNVVAAFIGQKDYAKAAWAIVKHLAKQGIKRNAFAIAGMLAWWAWQC---
>UniRef100_UPI000F652D05	50	0.250	9.887E-04	63	134	136	111	182	185
---------------------------------------------------------------RDTGEFLSCMKGKASDDLKSIFDVNAIAVLIGQEKYGEAAVAAVKYLAKQGIKRNAAGIAGVLLFYAAKCSI-
>UniRef100_UPI00125CB70C	46	0.225	2.995E-02	53	132	136	97	176	181
-----------------------------------------------------NELTTDTTSFRGAPQFLSCMKSKVSNDLKSIFNINAIASLIGQQKYYEAAVKAVQYLAKQGIKRNVAGIAGALAFYGAKC---
>UniRef100_UPI0009761044	44	0.285	1.404E-01	67	132	136	108	174	176
-------------------------------------------------------------------SFWSCMKNQLLDmigyQTFQALLRGGIQGLIQRKAWKAAAKLLIRYL---GDGIGVGFIAAQLAWYAIRC---
>UniRef100_V9W2Y6	44	0.311	1.911E-01	72	132	136	0	56	60
------------------------------------------------------------------------MKDKLMQEFEELVNIGTLGVLIAEKKWTELAWTLIKK----GVKRNPWVLAGWLAWQAAKC---
>UniRef100_UPI001F1C6CF6	43	0.252	3.541E-01	70	132	136	53	115	120
----------------------------------------------------------------------SCMEGKVEDDVKEIFHVNAIAVLIGQEKYFEAAEEAVAYLAKQGVKRNAAAIASVLAFYGARC---
>UniRef100_UPI001F2B8987	43	0.252	3.541E-01	70	132	136	116	178	183
----------------------------------------------------------------------SCMEGKVEDDVKEIFHVNAIAVLIGQEKYFEAAEEAVAYLAKQGVKRNAAAIASVLAFYGARC---
>UniRef100_A0A2S0WEI1	42	0.228	4.819E-01	63	132	136	52	121	129
---------------------------------------------------------------RSVGEFLTCMRSEVTDDLKSIFDINAIAALIGREKYWEAAVEAVKFLAKQGIKRNVAGLAATLAFYGGKC---
>UniRef100_A0A9D1RZL8	39	0.238	5.627E+00	66	132	136	112	178	183
------------------------------------------------------------------GEFLSCIKGKVDAEVKAIFDVNAIAVLIGQENYFKAAKKAVDFLARQGIRKNAVGIAAVLAYHGARC---
>101
SDQLEDSEVEAVAKGLEEXYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAXISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>U6S4W9	158	0.985	3.319E-41	0	135	136	55	190	191
SDQLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>uvig_117295_3	158	0.985	3.319E-41	0	135	136	55	190	191
SDQLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>A0A1C0USB4	156	0.557	1.610E-40	4	134	136	45	171	173
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPYD----RVPWDEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>MGYP000967978327	156	0.557	1.610E-40	4	134	136	45	171	173
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPYD----RVPWDEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>R2N7C7	143	0.686	7.398E-36	1	134	136	35	168	170
-EQIKNSEVDTVAQGLEQMFSNGVSEENFKNYVNANFSSEEITKSEKELDVNLSNTSSPIQARVNWNGLGQCMANKIKDEFFAMINVGAIVAAAQKKAWKELAMTVLIFAKANGLKTNALIVAGQLAVWAVQCGL-
>A0A2A5L654	137	0.286	1.156E-33	10	134	136	55	183	186
----------AVAKELEKMFVDGDIENLNIDYLIAKYGKDEIQATERFIGISENEsrifPTEHRIVQRDLADIGNCMLGKLGEEIRSMVNVNTIVAYIDKKLWLEAAKAIVAKVAAQGIKRNAAVMATVLAWYAVQCGL-
>3300014513.a:Ga0169742_10090_9	137	0.286	1.156E-33	10	134	136	55	183	187
----------AVAKELEKMFVDGDIENLNIDYLIAKYGKDEIQATERFIGISENEsrifPTEHRIVQRDLADIGNCMLGKLGEEIRSMVNVNTIVAYIDKKLWLEAAKAIVAKVAAQGIKRNAAVMATVLAWYAVQCGL-
>A0A256LDH3	134	0.503	1.053E-32	5	134	136	43	181	185
-----DPEVAVVAQELEKIFANGVSQENLNRYVLKNFSNKELTVAEKELDVNYN--PFSLQSKNDnnslhsvsvygWNNLGQCMYNKIKDEFFAMVNIGVIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVAAQLAVWAVQCGM-
>Q6DRR6	128	0.466	8.748E-31	4	134	136	44	176	179
----EQRQIDEVAAVLEKMFADGVTEENLKQYAQANYSEEELIIADNELNTNLSQIQDenAIMYKVDWGALGNCMANKIKDELLAMISVGTIIKYAQKKAWKELAKIVIKYVAKAGVKTNAALIAGQLAIWGLQCGI-
>MGYP000869820508	128	0.237	1.199E-30	4	133	136	55	192	196
----EEQQVEEVAKLLEKM-DKAPSREEAQRVLEDNFSQEELNGVAEELDIssdevltgaDAKDVGDAAATEEGFKDFYRCIKSKAGKDLRSALNVNAVMAAFGQKDYLKAAREIVKYLLKNGLKRNVFALAVTLGWYALQCS--
>MGYP000215453753	126	0.222	5.813E-30	4	132	136	51	184	188
----EKQQVEQLAKVLEKV-DNAPNREEAERIMVENFGEEGLEEAAKEIGVDPNNplgegsPTRSARGEVNEEGFVECMAGKVGDEIKGLLNINGVALALGQKDYPKMAAEIVKYLAKQGIKRNVGALALLLAWYGAQC---
>A0A0T8EFX5	116	0.504	2.127E-26	19	127	136	1	105	106
-------------------FRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPYD----RVPWDEMGGCIVGKIREDFFAMTNVSLIVKYAQKRAWLELAKVVLRFVKANRLKTNIYIIAGQLAF--------
>MGYP000013645973	116	0.244	2.127E-26	1	132	136	16	154	158
-EQIQDSqnQVDKVAKEFEDLFTNGiqINENSYainSDYLVQNYSSEEISGIVSLIEEISLINNTYTRSKRDIGSFLVCMKDKAVGDLKDMFNVGKFLVFIKTKAWKQAAEFAVKWLAKNGVKRNAVATAALLGWYGVQC---
>MGYP000875481199	113	0.232	1.936E-25	4	132	136	52	192	197
----EEQQVEEIAKFLEQV-DKAPSEKDAHRILEERFSQEELNGAAEELGVSPEEmfadggaqdgaklPEDPANNEESFGSFMRCIAGKAGNDLKSVIYSKDVLRAISKKQYKKAALAIVRHLAKQGIKRNIVVLGIQLGWYAFQC---
>MGYP000964187490	103	0.221	5.146E-22	5	103	136	30	133	134
-----QTEIESVASEFEQLFTKGIIisGNNYTfnhDYLTNNYTSDEIQAFIHLMDSTDLSPTFSKRRKRSIGSFAVCMKDKAVSDIADMFKVGAFVSFVQRKAWKEAAK--------------------------------
>A0A0K0G7I1	94	0.202	7.246E-19	0	132	136	10	172	176
SDDVEGAELAKLEQDLEFLFEEATTEEGDKYVLdeekaKDYFGEEnlpEIKILIKLINeeevteeefLNagiVPKHESYEEAKaevRDQNdnlvsnySWLGCMKTKIIAGTGLGFFGKGIDKLIEDKNWKKLSKEIIKIVGKNAVRGGAVGLAASLAVWSTMC---
>W7RBR4	91	0.250	1.235E-17	61	132	136	114	185	189
-------------------------------------------------------------SQFHTASWWGCLKEKIIDFTGLGFIGGGLEKMLKKKLWKKAATEIIKIVGKNAIRGGVLGLAGSLAWFSVRC---
>MGYP001084790427	85	0.971	1.389E-15	67	135	136	9	77	78
-------------------------------------------------------------------SLGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>MGYP001008199351	84	0.710	1.903E-15	66	134	136	15	83	85
------------------------------------------------------------------DEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>MGYP001068281740	75	0.984	3.611E-12	0	63	136	34	97	115
SDQLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARF------------------------------------------------------------------------
>MGYP000904613324	74	0.242	4.944E-12	67	132	136	14	79	83
-------------------------------------------------------------------SWTSCIKEKIMIATGIGFITGGMNKLIEEKAWKKLSLEIAKIVGKNAIKGGVVGLTASLAVWSIVC---
>MGYP000968663296	70	0.396	1.143E-10	4	56	136	64	116	117
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALAENKLETNINNLY-------------------------------------------------------------------------------
>A0A133S201	70	0.378	2.141E-10	19	92	136	1	70	72
-------------------FRNGVNEKNFTECVYKNFSQKDIALAENKLETNINNLYD----RVPWDEMGGCIARKIREEFFAMTNVSLTVRY-------------------------------------------
>MGYP001149482190	69	0.678	4.009E-10	79	134	136	22	77	81
-------------------------------------------------------------------------------EFFAMVNIGLIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVAAQLAVWAVQCGM-
>A0A0Q9UUB0	68	0.270	1.027E-09	38	132	136	83	175	203
--------------------------------------QQEIESSSCELaETTVINPIQPLAAKKSWKS---CMVASLKDHFGVALIevamTGGLWAYLQKKAYKEAAKLLIKI----GIGGNAIGLAATLTWYGTRC---
>A0A192YJB0	67	0.271	1.406E-09	67	132	136	117	183	185
-------------------------------------------------------------------SFWSCMKNQLLDmisyQTFQALLRGGIQGLIQRKAWKAAAKLLIRYLG-DGI--GVGFIAAQLSWYAIRC---
>MGYP000655452331	66	0.928	2.632E-09	80	135	136	7	62	63
--------------------------------------------------------------------------------IRDRISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>R8G7N7	62	0.238	8.256E-08	8	132	136	43	187	190
--------VEKLAEDLEFIMEKAAIRNsNDKVIdfdfdkLENRFGKlQEFKILKNEINndklnkheinqkqcTNISN-YNSLQAK-SWNGWKSCMVDALKDHFGVKIieiaFEGGLWGYLEKKAYKEAAKLLVKI----AVGSNVLGVTSFLVYYGAKC---
>UniRef100_A0A1X7GMK4	61	0.216	1.544E-07	7	132	136	38	178	206
-------QVDELAEDLEFLMEEAaiyDSENNVVgfhfDKLEDRFGeveelkmlQQEIESSSCELvetTTTETNSVQPLAAKKTWK---GCMIDSLKDHFGVAIIevamTGGLWAYLEQKAYKEAAKLLIKI----GVGGNVIGLAAFLTYYSAKC---
>A0A1X7GMK4	61	0.216	1.544E-07	7	132	136	38	178	206
-------QVDELAEDLEFLMEEAaiyDSENNVVgfhfDKLEDRFGeveelkmlQQEIESSSCELvetTTTETNSVQPLAAKKTWK---GCMIDSLKDHFGVAIIevamTGGLWAYLEQKAYKEAAKLLIKI----GVGGNVIGLAAFLTYYSAKC---
>MGYP001088613770	60	0.318	3.946E-07	5	70	136	29	92	98
-----DPEVAVAAQELEKIFVDAVSQKNLNRYALKNFSNKELTVAEKELDVNYN--PFSLQSKNDNNSLHS-----------------------------------------------------------------
>A0A1L6ZGN2	59	0.239	7.374E-07	66	132	136	107	173	182
------------------------------------------------------------------GTWGDCMIDSLKDHFGVAMIevalTGGLWSYLEKKAYKEAAKLLLKI----GIGGNVIGLVAFLTWYSAKC---
>A0A1J9VKJ1	55	0.252	1.674E-05	62	132	136	108	176	182
--------------------------------------------------------------KRS--AWTDCMVDAIKDHFGVAAVTaaleGGLWAYLEKKAYKEAAKLLVKF----AIGSNAVGLAGTLVYYGGKC---
>F7SG70	55	0.318	3.123E-05	5	70	136	24	87	93
-----DPEVAIAAQELEKIFVDVVSQKNLNRYALKNFSNKELTVAEKELDVNYN--PFSLQLKNDNNSLHS-----------------------------------------------------------------
>A0A0V8JI20	51	0.273	5.147E-04	64	132	136	107	175	181
----------------------------------------------------------------SNEAWKSCMIGAIKDHFGVAMVTaaleGGLWAYLEKKAYKEAAKLLVKF----AVGSNAVGLAGTLIYYGGVC---
>MGYP000101651737	46	0.236	2.903E-02	15	69	136	9	58	59
---------------LEQEFLISLI-KDKPETVYKNFSKKDIALAENKLEININNLYD----RVPWDEMG------------------------------------------------------------------
>A0A270AIV9	44	0.276	9.993E-02	86	132	136	1	43	71
--------------------------------------------------------------------------------------TGGLWANLEKKAYKEAAKLLVKI----GIGGNAIDLASFLTWYSARC---
>MGYP001000650989	44	0.276	1.853E-01	86	132	136	18	60	67
--------------------------------------------------------------------------------------TGGLWGYLKKKAYKEAAKLLVKI----GVGTNAATLAATLIYYGGKC---
>MGYP001053269802	44	0.216	1.853E-01	68	133	136	0	82	88
--------------------------------------------------------------------FGKCILNKMdlgeLKNLAKIIFKPATVRYLKSHAWKKASammvNAIVQYAPKKvaslAVKKFANLTLpgvgwASVAWWGAQCG--
>MGYP000578092484	42	0.315	8.644E-01	63	132	136	95	167	172
---------------------------------------------------------------RSPYDFGKCI---LKDYFGVYIDlvqgklWDSFVGYLQSEAWTEAAKIILKIIGKSASKANLIATAGQLALAAFNC---
